In [1]:
from transformers import AutoModelForCausalLM, AutoProcessor, GenerationConfig
from PIL import Image
import requests

/home/dnth/mambaforge-pypy3/envs/molmo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the processor
processor = AutoProcessor.from_pretrained(
    # 'allenai/Molmo-7B-D-0924',
    'molmo_7b_d_0924',
    trust_remote_code=True,
    torch_dtype='auto',
    device_map='auto'
)

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    # 'allenai/Molmo-7B-D-0924',
    'molmo_7b_d_0924',
    trust_remote_code=True,
    torch_dtype='auto',
    device_map='auto'
)

Loading checkpoint shards: 100%|██████████| 7/7 [00:11<00:00,  1.62s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


In [4]:
image_urls = [
    "sample_images/0a6ee446579d2885.jpg",
    "sample_images/0a8caaad03cfd733.jpg",
    # Add more image URLs as needed
]

prompts = [
    "Describe this image.",
    "What do you see in this picture?",
    # Add more prompts corresponding to each image
]

# Load images
images = [Image.open(url) for url in image_urls]

In [6]:
batch_inputs = processor.process(images=images, text=prompts)

# Move inputs to the correct device and make a batch
batch_inputs = {k: v.to(model.device) for k, v in batch_inputs.items()}

TypeError: can only concatenate str (not "list") to str

In [9]:
from transformers import AutoModelForCausalLM, AutoProcessor, GenerationConfig
from PIL import Image
import requests
import torch

# Load processor and model (same as before)
processor = AutoProcessor.from_pretrained(
    # 'allenai/Molmo-7B-D-0924',
    'molmo_7b_d_0924',
    trust_remote_code=True,
    torch_dtype='auto',
    device_map='auto'
)

model = AutoModelForCausalLM.from_pretrained(
    # 'allenai/Molmo-7B-D-0924',
    'molmo_7b_d_0924',
    trust_remote_code=True,
    torch_dtype='auto',
    device_map='auto'
)

# Create a batch of images and prompts
image_urls = [
    "https://picsum.photos/id/237/536/354",
    "https://picsum.photos/id/238/536/354",
    "https://picsum.photos/id/239/536/354"
]

images = [Image.open(requests.get(url, stream=True).raw) for url in image_urls]
prompts = ["Describe this image."] * len(images)

# Process each image-prompt pair and collect inputs
batch_inputs = []
for img, prompt in zip(images, prompts):
    inputs = processor.process(
        images=[img],
        text=prompt
    )
    batch_inputs.append({k: v.to(model.device) for k, v in inputs.items()})

# Create batched tensors
batched_inputs = {
    k: torch.stack([inputs[k] for inputs in batch_inputs])
    for k in batch_inputs[0].keys()
}

# Generate outputs for the entire batch
outputs = model.generate_from_batch(
    batched_inputs,
    GenerationConfig(max_new_tokens=200, stop_strings="<|endoftext|>"),
    tokenizer=processor.tokenizer
)

# Process generated outputs
generated_texts = []
for i, output in enumerate(outputs):
    generated_tokens = output[batched_inputs['input_ids'].size(1):]
    generated_text = processor.tokenizer.decode(generated_tokens, skip_special_tokens=True)
    generated_texts.append(generated_text)

# Print results
for i, text in enumerate(generated_texts):
    print(f"\nImage {i + 1} description:")
    print(text)


Loading checkpoint shards: 100%|██████████| 7/7 [00:00<00:00,  8.60it/s]



Image 1 description:
 This image captures a young black Labrador puppy, likely around six months old, sitting on a weathered wooden deck. The puppy's sleek, short fur is entirely black, including its nose, eyes, and ears, which are slightly floppy. The dog is positioned in the center of the frame, looking up directly at the camera with a curious and attentive expression. Its front paws are visible, with one slightly tucked under its body, while its back paws are hidden from view. The wooden deck beneath the puppy is made of light brown planks with visible knots and signs of wear, adding a rustic charm to the scene. The overall composition is simple yet striking, with the puppy's glossy black coat contrasting beautifully against the light wooden background.

Image 2 description:
 This black and white photograph captures a dense urban landscape, likely New York City, characterized by a multitude of skyscrapers and high-rise buildings. The image, taken from a high vantage point, possibly

In [11]:
# Generate outputs for the entire batch
outputs = model.generate_from_batch(
    batched_inputs,
    GenerationConfig(max_new_tokens=20, stop_strings="<|endoftext|>"),
    tokenizer=processor.tokenizer
)

# Process generated outputs
generated_texts = []
for i, output in enumerate(outputs):
    generated_tokens = output[batched_inputs['input_ids'].size(1):]
    generated_text = processor.tokenizer.decode(generated_tokens, skip_special_tokens=True)
    generated_texts.append(generated_text)

# Print results
for i, text in enumerate(generated_texts):
    print(f"\nImage {i + 1} description:")
    print(text)


Image 1 description:
 This image captures a young black Labrador puppy, likely around six months old, sitting on a weathered

Image 2 description:
 This black and white photograph captures a dense urban landscape, likely New York City, characterized by a multitude

Image 3 description:
 In this detailed photograph, a person's hand, likely a woman's due to the red nail polish
